## How to run the script

1. Please run the pre intervention script before the post intervention script
2. Define what time point to observe benefit tracking in select_periods

In [1]:
# time point 
######### change these numbers to what you want 
select_year = '2019-2020'
selec_period_final = 4
######### change these numbers to what you want 

#file names need to be in the proper naming conventions 


In [2]:
# define names 

select_periods = []
period_list=[]
for i in range(1,selec_period_final+1):
    tmp = 'P0{pp}'.format(pp=i)
    period_list.append(tmp)
    tmp2 = '{yy} {pp}'.format(yy=select_year,pp=tmp)
    select_periods.append(tmp2)

output_file_name = 'BT_LOSC_KPIs_TD_Pre_{yy}_{pp1}-{pp2}.csv'.format(yy = select_year, pp1 = period_list[0],pp2 = period_list[-1])
column_name = 'Pre-intervention ({yy} {pp1}-{pp2})'.format(yy = select_year, pp1 = period_list[0],pp2 = period_list[-1])

## Purpose of script

The aim of this script is to show a reduction in spending on LOSCs through delivering more jobs using in-house resources using the strategic framework for benefit and assessment stratergy. This will done in two steps:

First stage indicators:
1. Total LOSC hiring measured in hours/ days.
1. LOSC hiring expressed as equivalent Full Time Employees (FTE).
1. Total teams’ headcounts including in-house workers and equivalent LOSC FTE. 
1. LOSC FTE as a proportion of the total team’s headcount.  
1. Number of completed jobs and required jobs.

Second stage indicators:
1. Total LOSC spending

All this will be answered for pre and post intervention with this script focusing on PRE-intervention data. The date of intervention is taken as September 2021.

1. Define connections to SQL tables and Azure Storage blobs
2. Pre-porcess the LOSC data
3. Derive LOSC roles from "Item Description" column.
4. Merge to WGS data
5. Merge LOSC with Competences held
6. Team Size
7. Add LOSC recommendations taken up to date to the data
8. LOSC KPIs

## 1. Get all file paths and read files

#### 1.A. Import relavent libraries

In [3]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Supress scientific notation
import numpy as np
from azure.storage.blob import BlobServiceClient
import os
import pyodbc
from io import StringIO, BytesIO
import io

#### 1.B. Define connections to SQL tables and Azure Storage blobs

In [4]:
# Define the connection to SQL database
#conn = pyodbc.connect( # SQL server
                       # name of SQL data base
                        # UID =
                       # Password
#                      ) # databasename

# To connect to Azure devp envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure test envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure staging envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure prod envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)


#### 1.C Define all the Functions used in the script

In [13]:
# Define a funtion to read csv files directly from azure data storage
def download_csv(BLOBNAME,dtype_var):
    blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
    blob_data = blob_client_instance.download_blob()
    with BytesIO() as f:
        blob_data.readinto(f)
        f.seek(0)
        data = pd.read_csv(f,dtype=dtype_var)   
    return data

# Define a funtion to read excel files directly from azure data storage
def download_excel(BLOBNAME,dtype_var):
    blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
    blob_data = blob_client_instance.download_blob()
    with BytesIO() as f:
        blob_data.readinto(f)
        f.seek(0)
        data = pd.read_excel(f,dtype=dtype_var)   
    return data
    
# Rename columns
def rename_column(table_name):
    table_name.columns = table_name.columns.str.replace(' ', '')
    table_name.columns = table_name.columns.str.lower()

# Save csv files and upload to azure blob storage
def save_outputs_azure_blobs(output_name1,output_name2,azure_environment):
    output_name1.to_csv(output_name2, index = False)
    # Create a blob client using the local file name as the name for the blob
    blob_client = azure_environment.get_blob_client(container='output', blob='static/benefitTracking/LOSC/{tmp2}'.format(tmp2=output_name2))
    # Upload the created file
    with open(output_name2, "rb") as data:
        blob_client.upload_blob(data,overwrite=True)

#### 1.D Read the paths to all files/download blobs/read SQL tables

In [6]:
# LOSC1 (AZURE BLOB)
losc1=  download_csv('static/benefitsBaseline/LOSCMaint/COOM PO Line Report  Baseline (Maint, WD, Ops).csv',str)
rename_column(losc1)

# LOSC2 (AZURE BLOB)
losc2=  download_csv('static/benefitsBaseline/LOSC/COOM PO Line Report Baseline.csv',str)
rename_column(losc2)

# LOSC roles mapping (SQL)
losc_roles = pd.read_sql_query('''select * from [dbo].[LOSCRolesMapping]''', conn)
rename_column(losc_roles)

# OrgPlus (AZURE BLOB)
orgplus=  download_excel('static/benefitsBaseline/orgPlus/OrgPlus Baseline.xlsx',None)
rename_column(orgplus)

# WGS (SQL)
wgs = pd.read_sql_query('''select * from [dbo].[workgroupsetBaseline]''', conn)
rename_column(wgs)

# LOSC roles to competence mapping (AZURE BLOB)
losc_competences = download_excel('static/benefitsBaseline/LOSCRoles/LOSC Roles - Competence Mapping Baseline.xlsx',None)
rename_column(losc_competences)

# Team Size (AZURE BLOB)
comp_teamsize =  download_csv('static/benefitsBaseline/teamSize/team_size_PreInter.csv',None)
rename_column(comp_teamsize)

# losc recomendation with trainning demand 
losc_trainning_demand =  download_csv('static/benefitsBaseline/trainning_delivered_recommendations/losc_recomen_course_trainning.csv',None)
rename_column(losc_trainning_demand)

## 2. Pre-process  LOSC data

2. Read and Pre-process  LOSC data
    1. Merge the two LOSC data sets and drop duplicates
    2. Drop rows which don't have a cost centre assigned to them
    3. Remove data where "Purchase Approval Status" column is COMPOSING and not APPROVED as these are assumed to still be not finalised.
    4. Remove data where "Cancelled Flag" is CANCELLED as these are assumed to have been cancelled.
    5. Convert numeric data columns to number format from strings
    6. Keep only the records where the measurement type is hours
    7. Only keep data from the selected periods and year

#### 2.C. Pre process data

In [7]:
# Merge the two datasets
losc = losc1.append(losc2, ignore_index=True)

# Drop duplicate records
losc = losc.drop_duplicates().reset_index(drop=True)
len(losc)

# Keep only non "COMPOSING" records
print(losc['purchaseapprovalstatus'].unique())
losc = losc.loc[losc['purchaseapprovalstatus'] != 'COMPOSING'].reset_index(drop=True)
losc = losc.loc[losc['purchaseapprovalstatus'] != 'PENDING APPROVAL'].reset_index(drop=True)
print(losc['purchaseapprovalstatus'].unique())
print(len(losc))

# Keep only non "CANCELLED" records
print(losc['cancelledflag'].unique())
losc = losc.loc[losc['cancelledflag'] != 'CANCELLED'].reset_index(drop=True)
#losc = losc.loc[losc['cancelledflag'] != 'NaN'].reset_index(drop=True)
print(losc['cancelledflag'].unique())
len(losc)

# Change dtype of Cost and Quantity columns to number from string
losc['receivedamount'] = losc['receivedamount'].astype(float)
losc['receivedquantity'] = losc['receivedquantity'].astype(float)

# Drop all records apart from where the measurement type is hours
losc = losc.loc[~losc['unitmeaslookupcode'].isin(['Each', 'Day', 'Week', 'Mile', 'Day(s)', 'Metres', 'Daily', 'Box', 'Weekly', 'Per', 'Litre', 'Bag', 'Tonne', 'Pack', 'Package'])]
len(losc)

['APPROVED' 'COMPOSING' 'PENDING APPROVAL']
['APPROVED']
137759
[nan 'CANCELLED']
[nan]


121699

#### 2.D. Only keep data from 2021-2022 P01, 2021,2022 P02

In [8]:
# Extract Period data
losc['Period'] = losc['startdateandtimeperiod'].apply(lambda x: x.split(" ")[1])

# Extract Year data
losc['Year'] = losc['startdateandtimeperiod'].apply(lambda x: x.split(" ")[0]) # gives 2020 - 2021

losc_time = losc.loc[losc['startdateandtimeperiod'].isin(select_periods)]
print(losc_time['startdateandtimeperiod'].value_counts())
len(losc_time)

2019-2020 P04    3813
2019-2020 P03    1273
2019-2020 P02     138
2019-2020 P01      72
Name: startdateandtimeperiod, dtype: int64


5296

## 3. Only keep competency-based roles

#### 3.A. Derive LOSC roles from "Item Description" column

In [9]:
# Rename Role column
losc_roles.rename(columns={'roletypeloscdata': 'roletype'}, inplace=True)

# Identify role types from the item description column in LOSC data
losc_time['roletype'] = 'Unknown'

for i in range(0,len(losc_roles['roletype'])):

    indx = losc_time['itemdescription'].str.contains(losc_roles["roletype"][i], regex=False, case=False)
    losc_time['roletype'] = losc_time['roletype'].mask(indx, losc_roles["roletype"][i])


<ipython-input-9-89dde703d10c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  losc_time['roletype'] = 'Unknown'
<ipython-input-9-89dde703d10c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  losc_time['roletype'] = losc_time['roletype'].mask(indx, losc_roles["roletype"][i])


#### 3.B. Merge the LOSC data which have the role type column to competences based/generic data

In [10]:
## Link the LOSC data to the role data to get the grouped (cleaned) roles and use these to then link to competences.
# Merge dataframes
losc_rt = losc_time.merge(losc_roles[['roletype', 'roletypecleaned', 'genericcompetencybasedrole']], how='left', on='roletype')

# Fill in unknown roles with Unknown
losc_rt['genericcompetencybasedrole'].fillna('Unknown', inplace=True)

## Roles are grouped by whether they are competence based or not. Let's see how many competence-based roles there are in the dataset.
grouped_losc_stats2 = pd.DataFrame(losc_rt.groupby('genericcompetencybasedrole')['purchaseordernumber'].count())
grouped_losc_stats2['percentage(%)'] = np.round((grouped_losc_stats2['purchaseordernumber'] / len(losc)) * 100, 1)
grouped_losc_stats2

,purchaseordernumber,percentage(%)
genericcompetencybasedrole,,
Competency-based,2582,2.100
Generic,2219,1.800
Unknown,495,0.400


#### 3.C. Only keep competency based roles

In [11]:
losc_rt = losc_rt.loc[losc_rt['genericcompetencybasedrole'] == 'Competency-based']


## 4. Assign LOSC requests to WGS

#### 4.A. Merge Work group set and OrgPlus

In [15]:
# drop all duplicates in orgplus first so should have single Parent UPN in orgplus. 
orgplus = orgplus.drop_duplicates().reset_index(drop=True)
orgplus = orgplus.loc[~orgplus['employee_number'].isna()]
#wgs['workgroupset'] = wgs['workgroupset'].str.replace(' ','')

# Merge Work group set and OrgPlus
WGS_OrgPlus = wgs.merge(orgplus[['parent_upn', 'cost_centre']], how='left', left_on='smengineerupn', right_on='parent_upn')

# Convert cost centre column to str
WGS_OrgPlus['cost_centre'] = WGS_OrgPlus['cost_centre'].astype(str)
# remove the .0 from the end of cost center 
WGS_OrgPlus["cost_centre"] = WGS_OrgPlus["cost_centre"].str.split('.').fillna('[]').str[0]

# Remove records where the cost centre code is missing as these can't be joined to a work group set
WGS_OrgPlus = WGS_OrgPlus[WGS_OrgPlus['cost_centre'] != 'nan']

# Drop duplicates
WGS_OrgPlus.drop_duplicates(inplace=True)

WGS_OrgPlus['workgroupset'].nunique()

1046

#### 4.B. How do we attribute LOSC spend when more than one SM links to the same Cost Centre code?

In [16]:
# Get the number of WorkGroup Sets linked to the same cost centre
linked_workgroup_sets = pd.DataFrame(WGS_OrgPlus.groupby('cost_centre')['workgroupset'].count())
linked_workgroup_sets = linked_workgroup_sets.rename(columns={'workgroupset': 'Work Group Set Counts'}).reset_index()

# Link the counts back to the dataset
WGS_OrgPlus_cnt = WGS_OrgPlus.merge(linked_workgroup_sets, how='left', on='cost_centre')
print(WGS_OrgPlus_cnt['workgroupset'].nunique())

1046


#### 4.C. Merge WGS_OrgPlus data to LOSC

In [19]:
# Join workgroup set to losc data
losc_rt_wgs = losc_rt.merge(WGS_OrgPlus_cnt, how='left', left_on=['costcenternumber'], right_on=['cost_centre'])

# Join workgroup set to losc data
losc_rt_wgs = losc_rt.merge(WGS_OrgPlus_cnt, how='left', left_on=['costcenternumber'], right_on=['cost_centre'])

# Drop all LOSC data that was not mapped to sections. 
losc_rt_wgs = losc_rt_wgs.loc[~losc_rt_wgs['masterdeliveryunit'].isna()]
print(losc_rt_wgs['workgroupset'].nunique())

272


#### 4.D. Adjust the cost and units used by the number of Sections attributed to each PO in the LOSC data.

In [20]:
# Divide by the number of Work Group Sets linked to each PO
# Uncomment this section if you want the cost to be equally split between the SMs
losc_rt_wgs["Adjusted Cost (by SM number)"] = losc_rt_wgs['receivedamount'] / losc_rt_wgs['Work Group Set Counts']
losc_rt_wgs["Adjusted Quantity (by SM number)"] = losc_rt_wgs['receivedquantity'] / losc_rt_wgs['Work Group Set Counts']

## 5. Merge LOSC requests with Competences mapping

#### 5.A. We can now link the losc data to the competences

In [21]:
# Merge dataframe
losc_merged_comp = losc_rt_wgs.merge(losc_competences, how='left', left_on='roletypecleaned', right_on='roletype')
losc_merged_comp.drop('roletype_y', axis=1, inplace=True)  # Drop column duplicated by the merge
losc_merged_comp.rename(columns={'roletype_x': 'roletype'}, inplace=True)  # Rename column duplicated by the merge

print(losc_merged_comp['workgroupset'].nunique())

272


#### 5.B. Since there are roles that have more than one competence linked to them, we need to find the number of competences for each role and divide the total cost to get the cost per competence for each purchase order (PO request).

In [22]:
# Group by PO number and Role as there might be multiple roles per PO
# There are cases where the same PO and Role is repeated for multiple periods of time hence the use of a lambda instead of a count
no_of_competences = pd.DataFrame(losc_merged_comp.groupby(['purchaseordernumber', 'roletype'])['parentcompetence'].apply(lambda x: x.unique().shape[0]).reset_index())
no_of_competences.rename(columns={'parentcompetence': 'Number of Linked Competences'}, inplace=True)

# Join the counts back to the original table
losc_merged_comp_cnt = losc_merged_comp.merge(no_of_competences, how='left', on=['purchaseordernumber', 'roletype'])
# To check the above you can query the "losc_merged_comp_cnt" dataframe for different Purchase Order Number
losc_merged_comp_cnt['workgroupset'].nunique()

272

#### 5.C. ASSUMPTION: We can now adjust the PO cost per competence. We will divide equally the cost by the number of parent competences linked to it.

In [23]:
# # Divide by the number of competences
losc_merged_comp_cnt["Adjusted Cost (£)"] = losc_merged_comp_cnt["Adjusted Cost (by SM number)"] / losc_merged_comp_cnt['Number of Linked Competences']
losc_merged_comp_cnt["Adjusted Quantity (hrs)"] = losc_merged_comp_cnt["Adjusted Quantity (by SM number)"] / losc_merged_comp_cnt['Number of Linked Competences']

In [24]:
# Remove entries where Linked Competence is na
losc_merged_comp_cnt = losc_merged_comp_cnt.loc[~losc_merged_comp_cnt['linkedcompetence'].isna()]
print(losc_merged_comp_cnt['workgroupset'].nunique())

# remove whitespaces from the columns 
losc_merged_comp_cnt['linkedcompetence'] = losc_merged_comp_cnt['linkedcompetence'].str.strip()


268


#### 5.D. Aggregate data

In [26]:
# Group by year and competence and role
losc_agg = \
losc_merged_comp_cnt[['routelookup','masterdeliveryunit', 'workgroupset','linkedcompetence',
            'Adjusted Quantity (hrs)','Adjusted Cost (£)']].groupby(['routelookup','masterdeliveryunit', 'workgroupset','linkedcompetence']).agg({'Adjusted Quantity (hrs)':'sum','Adjusted Cost (£)':'sum'})

losc_agg.reset_index(inplace=True)  # Reset the index
losc_agg.rename(columns={'linkedcompetence':'Competence Alias'},inplace=True)
print(losc_agg['workgroupset'].nunique())

268


## 6. Merge Team Size

#### 6.A. Calculate the number of hours worked per year by teams

In [27]:
comp_teamsize['Hours worked per year'] = comp_teamsize['teamsize']*1610
#comp_teamsize = comp_teamsize.drop('unnamed:0',axis=1)
#comp_teamsize['workgroupset'] = comp_teamsize['workgroupset'].str.replace(' ','')

#comp_teamsize = comp_teamsize.rename(columns={'masterdeliveryunit':'master-deliveryunit'})
print(comp_teamsize['workgroupset'].nunique())


1045


#### 6.B Merge team size with LOSC data

In [28]:
# Merge competent team and losc year 1 and 2 data 
losc_agg_comp_team = pd.merge(losc_agg,comp_teamsize, 
                              left_on=['masterdeliveryunit','workgroupset'],
                              right_on=['masterdeliveryunit','workgroupset'])
print(losc_agg_comp_team['workgroupset'].nunique())
losc_agg_comp_team.head(5)

268


,routelookup,masterdeliveryunit,workgroupset,Competence Alias,Adjusted Quantity (hrs),Adjusted Cost (£),teamsize,Hours worked per year
0,Anglia,Ipswich,Colchester SM(OLE),ES,10.000,305.000,30,48300
1,Anglia,Ipswich,Colchester SM(SIGNALS),ES,8.000,221.600,29,46690
2,Anglia,Ipswich,Ipswich Depot IME,COSS,332.000,5630.240,8,12880
3,Anglia,Ipswich,Ipswich Depot IME,COSS CRP LLT,332.000,5630.240,8,12880
4,Anglia,Ipswich,Ipswich Depot IME,COSS OLP,332.000,5630.240,8,12880


## 7. Merge LOSC trainning delivered recommendations

In [29]:
# NEW METHOD
#losc_trainning_demand['workgroupset'] = losc_trainning_demand['workgroupset'].str.replace(' ','')
losc_trainning_demand = losc_trainning_demand.rename(columns={'competencealias':'Competence Alias'})
print(losc_trainning_demand['workgroupset'].nunique())
#osc_trainning_demand

## Merge losc data with recommendations
losc_agg_comp_team_rec = pd.merge(losc_agg_comp_team,losc_trainning_demand,how='left',
                                    left_on = ['masterdeliveryunit','workgroupset','Competence Alias','routelookup'],
                                    right_on = ['masterdeliveryunit','workgroupset','Competence Alias','routelookup'])
losc_agg_comp_team_rec = losc_agg_comp_team_rec.loc[~losc_agg_comp_team_rec['recommendednumberofpeopletotrain'].isna()]

## Number of unique work group sets in pre and post inervention 
print(losc_agg_comp_team_rec['workgroupset'].nunique())
losc_agg_comp_team_rec.head(3)

53
15


,routelookup,masterdeliveryunit,workgroupset,Competence Alias,Adjusted Quantity (hrs),Adjusted Cost (£),teamsize,Hours worked per year,coursetitle,recommendednumberofpeopletotrain,losccostbenefit(£),employeenumber
139,Anglia,Romford,PITSEA SM (OLE),PC,37.000,1026.120,28,45080,Protection Controller,1.000,4738.000,1.000
441,East Midlands,Bedford,Bedford SM(SIGNALS),SMTH,176.000,7888.320,25,40250,Failure and Incident Testing (Level 1),1.000,175900.000,2.000
442,East Midlands,Bedford,Bedford SM(SIGNALS),SMTH,176.000,7888.320,25,40250,Failure and Incident Testing (Level 2),1.000,175900.000,1.000


## 8. LOSC KPIs

#### 8.A. Work group Set and Competence level

In [32]:
# group by Work group set and sum quantity recieved 
tmp2 = losc_agg_comp_team_rec.groupby(['routelookup','masterdeliveryunit','workgroupset','Competence Alias'])[['Adjusted Quantity (hrs)','Adjusted Cost (£)']].sum().reset_index()
tmp3 = losc_agg_comp_team_rec.groupby(['workgroupset'])['teamsize'].max().reset_index()

losc_final = pd.merge(tmp2,tmp3,left_on=['workgroupset'],right_on=['workgroupset'])
losc_final = losc_final.rename(columns={'masterdeliveryunit':'Master Delivery Unit','routelookup':'Route'})
losc_final['1.2 LOSC head count'] = losc_final['Adjusted Quantity (hrs)']/1610
losc_final = losc_final.rename(columns={'Adjusted Quantity (hrs)':'1.1 LOSC hiring (Hrs)','Adjusted Cost (£)':'2.1 LOSC spending (£)','teamsize':'Team Size','workgroupset':'Work Group Set'})

# LOSC KPIs
losc_final['1.3 Team head count: FTE+LOSC'] = losc_final['Team Size']+losc_final['1.2 LOSC head count']
losc_final['1.4 Proportion of LOSC as total team headcount'] = losc_final['1.2 LOSC head count']/(losc_final['Team Size']+losc_final['1.2 LOSC head count'])*100
losc_final.head(3)

,Route,Master Delivery Unit,Work Group Set,Competence Alias,1.1 LOSC hiring (Hrs),2.1 LOSC spending (£),Team Size,1.2 LOSC head count,1.3 Team head count: FTE+LOSC,1.4 Proportion of LOSC as total team headcount
0,Anglia,Romford,PITSEA SM (OLE),PC,37.000,1026.120,28,0.023,28.023,0.082
1,East Midlands,Bedford,Bedford SM(SIGNALS),SMTH,352.000,15776.640,25,0.219,25.219,0.867
2,Kent,London Bridge,London Bridge Central SM(TRACK),COSS CRP LLT,42.667,733.013,20,0.027,20.027,0.132


In [33]:
# The format needs to change to match what Kabita requires for creating the pipelines in SQL 
losc_final_SQL = pd.melt(losc_final, id_vars=['Route','Master Delivery Unit', 'Work Group Set','Competence Alias','Team Size'], var_name='KPIs', value_name='KPI value')
losc_final_SQL.head(5)

,Route,Master Delivery Unit,Work Group Set,Competence Alias,Team Size,KPIs,KPI value
0,Anglia,Romford,PITSEA SM (OLE),PC,28,1.1 LOSC hiring (Hrs),37.000
1,East Midlands,Bedford,Bedford SM(SIGNALS),SMTH,25,1.1 LOSC hiring (Hrs),352.000
2,Kent,London Bridge,London Bridge Central SM(TRACK),COSS CRP LLT,20,1.1 LOSC hiring (Hrs),42.667
3,Kent,London Bridge,London Bridge Central SM(TRACK),ES,20,1.1 LOSC hiring (Hrs),41.333
4,Kent,London Bridge,London Bridge Central SM(TRACK),LB 3rd-R St-i,20,1.1 LOSC hiring (Hrs),457.000


In [35]:
# save output as csv and upload to azure blob storage containers
save_outputs_azure_blobs(losc_final_SQL,output_file_name,blob_service_client_instance_devp)
save_outputs_azure_blobs(losc_final_SQL,output_file_name,blob_service_client_instance_test)
save_outputs_azure_blobs(losc_final_SQL,output_file_name,blob_service_client_instance_prod)
save_outputs_azure_blobs(losc_final_SQL,output_file_name,blob_service_client_instance_staging)


#### 8.B. Summary of KPIs

In [36]:
data={'KPIs':['No. of WGS in LOSC recommendations',
              'No. of WGS found in LOSC data',
              '1.1 Total LOSC hiring measured in hours/ days',
              '1.2 LOSC hiring expressed as equivalent Full Time Employees (FTE)',
              '1.3 Total teams’ headcounts including in-house workers and equivalent LOSC FTE',
              '1.4 LOSC FTE as a proportion of the total team’s headcount',
              '1.5 Number of completed jobs and required jobs',
              '2.1 Total LOSC spending'],
      column_name:[losc_trainning_demand['workgroupset'].nunique(),
                          losc_agg_comp_team_rec['workgroupset'].nunique(),
                         losc_final['1.1 LOSC hiring (Hrs)'].sum(),
                         losc_final['1.1 LOSC hiring (Hrs)'].sum()/1610,
                         losc_final['Team Size'].sum()+losc_final['1.2 LOSC head count'].sum(),
                         losc_final['1.2 LOSC head count'].sum()/(losc_final['Team Size'].sum()+losc_final['1.2 LOSC head count'].sum())*100,
                         '',
                         losc_final['2.1 LOSC spending (£)'].sum()]}

LOSC_KPIs=pd.DataFrame(data)
LOSC_KPIs

,KPIs,Pre-intervention (2019-2020 P01-P04)
0,No. of WGS in LOSC recommendations,53
1,No. of WGS found in LOSC data,15
2,1.1 Total LOSC hiring measured in hours/ days,3080.167
3,1.2 LOSC hiring expressed as equivalent Full T...,1.913
4,1.3 Total teams’ headcounts including in-house...,487.913
5,1.4 LOSC FTE as a proportion of the total team...,0.392
6,1.5 Number of completed jobs and required jobs,
7,2.1 Total LOSC spending,108114.860


In [37]:
# read output to csv
LOSC_KPIs.to_csv('BT_LOSC_KPIs_TD_PrePost_Summary.csv', index = False)

#### 8.C. Data info

In [38]:
data={'':['LOSC',
              'OrgPlus+WGS',
              'competences',
              'teamsize',
              'Trainning delivered and recommendations',
              'LOSC time',
              'LOSC time + role type',
              'LOSC time + role type + (OrgPlus+WGS)',
              'LOSC time + role type + (OrgPlus+WGS) + competences',
              'LOSC time + role type + (OrgPlus+WGS) + competences + teamsize',
              'LOSC time + role type + (OrgPlus+WGS) + competences + teamsize + trainning delivered'],
              'WGS':['-',
              WGS_OrgPlus['workgroupset'].nunique(),
              '-',
              comp_teamsize['workgroupset'].nunique(),
              losc_trainning_demand['workgroupset'].nunique(),
              '-',
              '-',
              losc_rt_wgs['workgroupset'].nunique(),
              losc_merged_comp_cnt['workgroupset'].nunique(),
              losc_agg_comp_team['workgroupset'].nunique(),
              losc_agg_comp_team_rec['workgroupset'].nunique()],     
              'length':[len(losc),
              len(WGS_OrgPlus),
              len(losc_competences),
              len(comp_teamsize),
              len(losc_trainning_demand),
              len(losc_time),
              len(losc_rt),
              len(losc_rt_wgs),
              len(losc_merged_comp_cnt),
              len(losc_agg_comp_team),
              len(losc_agg_comp_team_rec)]}
              
losc_data_info=pd.DataFrame(data)
losc_data_info

,,WGS,length
0,LOSC,-,121699
1,OrgPlus+WGS,1046,1943
2,competences,-,157
3,teamsize,1045,1045
4,Trainning delivered and recommendations,53,65
5,LOSC time,-,5296
6,LOSC time + role type,-,2582
7,LOSC time + role type + (OrgPlus+WGS),272,2538
8,LOSC time + role type + (OrgPlus+WGS) + compet...,268,15139
9,LOSC time + role type + (OrgPlus+WGS) + compet...,268,2710
